# Exclude Outlier Users

This notebook filter out users who spent too little time or too much time on a specified course. Filtering is done using IQR.

# 1. Initialize paths and file names

# 2. Load event data

# 3. Save list of outlier users